In [1]:
import numpy as np
from pylab import*
from scipy.integrate import *
import pandas as pd
from numpy import linalg
pd.set_option('max_rows', 5)
%matplotlib inline

# Funciones utilizadas:

## Para encontrar las cuando ocurren y la amplitud de las ondas P, QRS y T.
### Para determinar si un punto coorresponde a una onda se encontraron los cuatro datos más grandes que eran mayores a los ```n=10``` numeros anteriores y posteriores a ellos. 

- Las funciones ```MenorVecinos(lis,n,v)``` y ```MayorVecinos(lis,n,v)``` se utiliza para determinar si en el número en la posición ```n``` de la lista ```lis``` es mayor que los ```v``` números anteriores en el caso de ```MenorVecinos(lis,n,v)``` y posteriores para ```MayorVecinos(lis,n,v)```. 

<br>

- La función ```Pico(lis,n,v)``` aplica las funciones ```MenorVecinos(lis,n,v)``` y ```MayorVecinos(lis,n,v)``` al número ```n``` de la lista ```lis``` para determinar si éste es un máximo local. Si se da el caso en que ```n```-```v```<0 entonces se ajusta el valor de ```v``` en la función ```MenorVecinos(lis,n,v)``` de tal forma que ```n```se compare hasta el index 0. Así mismo si ```n```+```v```>```len(lis)-1``` se ajusta el valor de ```v```en la función ```MayorVecinos(lis,n,v)``` para que el número se compare hasta el número con index ```len(lis)-1```.

<br>

- La función ```Picos(lis,n,v)``` encontrar a los ```n``` números màs grandes de la lista ```lis```que cumplan con los criterio de la función ```Pico```
<br>


### Las señales de la base de datos empiezan en complejo QRS, y terminan poco después del siguiente complejo QRS. ya que solo nos intereza lo que pasa entre estas dos ondas, se creó una función que detecta a los dos "picos" más grandes, y regresa una lista que contiene los mismos valores para los números que se encuentran entre los dos picos y sustituye al resto por NaN.

<br>

- La función Cortar(ECG) recibe una lista o un array ```ECG``` y aplica la función Picos(ECG,2,10) para encontrar a las dos complejos QRS y regresar una lista un array que contiene a ECG[0:QRS] , donde QRS es el índice del segundo complejo QRS. Si no se encuentran dos picos, entonces regresa array que contiene 187 NaNs.

<br>

- La función PicosXY(lisX,lisY,n,v) utiliza a la función Picos(ECG,n,v) con ECG=lisY y regresa los valores donde encuentra los picos en lisY y los valores que le corresponden a lisX.

<br>

- La función Tiempo(lis) regresa array con los valores de tiempo teniendo en cuenta que los valores de lis fueron tomados a una frecuencia de  150 Hz.

<br>

- La función ```PicosYT(lisY,n=4,v=10)```regresa los valores de ```PicosXY(lisX,lisY,n,v)```con  ```lisX```=```Tiempo(lis)```.

<br>

- La función ```PicosYTDF(lisY,n=4,v=10)``` se utiliza para aplicarle a una serie la función ```PicosYT(lisY,n=4,v=10)``` y finalmente es usada por ```PicosDF(ECGS)``` para encontrar los picos de todas las filas de DataFrame ```ECGS```.


In [2]:
#Si lis es lista/array de números la función dice si el número con index n es mayor que
#los n v números anteriores.
def MenorVecinos(lis,n,v):
    for a in range(v):
        a=a+1
        if lis[n]<=lis[n-a]:
            return(False)
    return(True)

#Si lis es lista/array de números la función dice si el número con index n es mayor que los n v números posteriores.
def MayorVecinos(lis,n,v):
    for a in range(v):
        a=a+1
        if lis[n]<=lis[n+a]:
            return(False)
    return(True)

#Si lis es lista/array de números la función dice si el número con index n es mayor que los n v números anteriores y 
#los v números posteriores.
def Pico(lis,n,v):
    vinf=v
    vsup=v
    if n<v:
        vinf=n
    if n+v>len(lis)-1:
        vsup=len(lis)-n-1
    
    if MayorVecinos(lis,n,vsup) and MenorVecinos(lis,n,vinf):
        return(True)
    else:
        return(False)
    
#Si lis es lista/array de números la función regresa el index de  los n números más grandes que cumplen que son
# mayores que los v anteriores y los v superiores.
#Si n es mayor a la cantidad de números que cumplen esto, solo regresa la cantidad de números que lo cumplen
#Hay concideraciones especiales cuando n-v<0 o n+v>len(lis)-1

def Picos(lis,n,v): 
    lis=list(lis)
    lis2=list(lis)
    X=[]
    n0=0
    m0=0
    while n0<n and m0<len(lis):
        Max=np.nanmax(lis2)
        if not isnan(Max):       
            imaximo=lis2.index(Max)
            if Pico(lis,imaximo,v):
                X.append(imaximo)
                n0=n0+1
            lis2[imaximo]=NaN
            m0=m0+1
        else:
            break
            
    return(np.array(X))

#Esta encuentra n máximos locales de lisY (usando criterios de la fun Picos) y regresa una lista con [x1,x2,...,xn]
#y la lista de [Y1, Y2,...,Yn] : Xi cumple es máximo local.

def PicosXY(lisX,lisY,n,v):
    X=[]
    Y=[]
    picos=Picos(lisY,n,v)
    for a in picos:
        X.append(lisX[a])
        Y.append(lisY[a])
    return(X,Y)


#Esta función regresa una array con el tiempo del ECG que se le de. La frec de muestreo es 125Hz
def Tiempo(lis):
    lis=list(lis)
    tiempo=np.linspace(0,len(lis)/125,len(lis))
    return(tiempo)

#Esta función recive un ECG y regresa un array con los valores desde pico R hasta el otro pico R
def Cortar(ECG):
    picos=Picos(ECG,2,1)
    if len(picos)==2:
        QRS=max(picos)
        return(ECG[0:QRS+1])
    else:
        return np.array([NaN]*187)

#Esta función hace lo mismo que la función picos, pero regresa una lista con los picos y la otra con tiempos.
def PicosYT(lisY,n=4,v=10):
    X=[]
    Y=[]
    tiempo=Tiempo(lisY)
    picos=PicosXY(tiempo,lisY,n,v)
    X=picos[0]
    Y=picos[1]
    
    dic={X[i]: Y[i] for i in range(len(X))}
    X.sort()
    Y=[]
    for a in X:
        Y.append(dic[a])
    

    return(X,Y)
    

#Grafica los picos
def GraficaPicos(ECG,n=4,v=10):
    ECGp=Cortar(ECG)
    tiempo=Tiempo(ECGp)
    X,Y=PicosXY(tiempo,ECGp,n,v)
    plot(tiempo,ECGp)
    plot(X,Y,".")
    
#Esta función hace lo mismo que PicosYT pero regresa solo una lista con los valores: (V0,T0,V1,T1,V2,T2,V3,T3)
def PicosYTDF(lisY,n=4,v=10):
    X=[]
    Y=[]
    tiempo=Tiempo(lisY)
    picos=PicosXY(tiempo,lisY,n,v)
    X=picos[0]
    Y=picos[1]
    
    dic={X[i]: Y[i] for i in range(len(X))}
    X.sort()
    Y=[]
    for a in X:
        Y.append(dic[a])

    Z=[]
    for a in range(len(X)):
        Z.append(X[a])
        Z.append(Y[a])
    Z=Z+[NaN]*(8-len(Z))
    
    return(Z)
    #return(X,Y)
    
    
#Esta función le aplica a un DF la función PicosYTDF y regresa cada dato en una columna
def PicosDF(ECGS):
    PICOS=pd.DataFrame()
    PICOS=ECGS.apply(PicosYTDF)
    PICOS=PICOS.to_frame()
    PICOS=pd.DataFrame(PICOS.iloc[:,0].tolist(), columns=["t0","v0","t1","v1","t2","v2","t3","v3"])
    #pd.concat([PICOS[:], pd.Series([0,1,2,3,4,5,6,7])], axis=1)
    #PICOS=PICOS.to_frame()
    #PICOS=ECGS.apply(lambda x:PicosYTDF(x), axis=1, result_type='expand')
    return(PICOS)



### Se realizó una función que le ajusta un polinomio de grado n a los ECGS para determinar si se pueden utiilizar como variables para clasificarlos:

<br>

- La función ```polifit(X,Y,n)``` regresa una lista 
 [$ \beta_0 , \beta_1 ,..., \beta_n  $] dónde $P_n = \beta_0 + \beta_1 x + \beta_2 x^2 +..+ \beta_n x^n$ es el polinomio de grado $n$ que mejor se ajusta al conjunto de de pares ordenados de las lis ```X```y ```Y```
 <br>
 
 -La función ```Betas(ECG,n=15)``` recibe una lista, array o serie ```ECG``` le quita todos los valores ```NaN``` y regresa el resultado de la función ```polifit(X,Y,n)``` con ```X```=```ECG```y ```Y```=```Tiempo(ECG)```.
 <br>
 
 -La función ```BetasDF(DF)``` le aplica la función ```Betas(ECG,n=15)``` a cada fila del DataFrame ```DF``` y en vez de regresar una lista, regresa cada $\beta$ en una columna. Ya que es probable que ```DF```tenga valores ```NaN``` se creó la función QuitaNan, la cual le quita a cada fila de ```DF```los valores ```NaN```antes de aplicar la funcion ```Betas```


In [3]:
#Esta función es para quitar los valores NaN de una señalm regresa una señal más corta pero
#sin NaN s.
def QuitaNan(V):
    lenV=len(V)
    for a in range(lenV):
        if np.isnan(V[a]):
            n=a
            break
        elif a==lenV-1:
            n=a
    if n==0:
        for b in range(len(V)):
            V[b]=0
        return(V)
    else:    
        V=V[:n]
        return(V)

#Esta función le ajusta a un conjunto de datos X y Y un polinomio de grado n y regresa una
#lista [b0,b1,...,bn] tal que el poli que mejor se ajusta es b0+b1 x+b2 x^2 +...+bn x^n

def polifit(X,Y,n):
    Xm=[]
    #for a in range(1,n+2):
    for x in X:    
        Xv=[1]
        for a in range(1,n+1):
        #for x in X:
            Xv.append(x**a)
        Xm.append(Xv)
    Xm=matrix(Xm)
    Yv=matrix(Y)
    Yv=Yv.transpose()
    return((linalg.inv(Xm.transpose()*Xm)*Xm.transpose()*Yv).transpose().tolist()[0])

#Esta función aplica la función polifit a (Tiempos(ECG),ECG) donde ECG puede ser una serie.
def Betas(ECG,n=15):
    ECG=list(ECG)
    Y=QuitaNan(ECG)
    X=Tiempo(Y)
    return(polifit(X,Y,n))
#Esta función le realiza polifit a los renglones de un DF. Regresa una serie
def BetasDF(DF):
    BETAS=DF.apply(Betas,axis=1,result_type='expand')
    return(BETAS)

## Primero extraemos la base de datos "mitbih_train.csv"
 ESGS es un DataFrame sin header que contiene a los datos de entrenamiento.

In [4]:
ECGS=pd.read_csv("mitbih_train.csv",header=None)

### Posteriormente se iguala a 0 el dato que cooresponde a la clasificación de latido para que no interfiera en la función cortar, ya que esta encuentra máximos.

In [5]:
for a in range(len(ECGS)):
    ECGS.iloc[a][187]=0

## Se aplica la función cortar a toda la base de datos y se guarda el DF en la variable ECGSC.
Para que cada fila del DF tenga la misma cantidad de números, los valores que se quitaron de la base original por estar después del segundo pico se convierten en NaNs.


In [6]:
# Se usa axis=1 para que la función Cortar se aplique a las filas y no a las columnas
ECGSC=ECGS.apply(Cortar,axis=1)

### Posteriormente aplica ```BetasDF(DF)``` con ```DF```=```ECGSC``` y  se guarda en la variable ```BETAS``` para ser posteriormente guardada como un archivo csv llamado "BetasTrain.csv"

In [7]:
BETAS=pd.DataFrame(BetasDF(ECGSC))

In [ ]:
BETAS.to_csv("BetasTrain.csv")

# Esta sección pretende utilizarse en un futuro para encontrar a las ondas P, QRS y T

#### La siguiente función aplica el método de media movil a una lista con la finalidad filtrar el "ruido" de alta grecuencia.

In [1]:
#Esta función aplica método media rodante a una lista, v es el parámetro de vecindad.
def Roll(lis,v): 
    lis=list(lis)
    lisZ=[]
    
    a=0
    while a<len(lis):
        if a<v:
            lisZ.append(lis[a])
        elif a+v>len(lis):
            lisZ.append(lis[a])
        else:
            lis2=[]
            
            for b in lis[a-v:a+v+1]:
                lis2.append(b)
            lisZ.append(mean(lis2))
        a=a+1
    return(lisZ)

In [ ]:
SECGSC=ECGSC.apply(Roll,axis=1,v=(2))

### Finlmente se guarda en la variable ```PICOS ``` los valores de tiempo y de amplitud de dónde se encontraron picos.

In [ ]:
PICOS=PicosDF(SECGSC)